In [1]:
import pickle
import torch
import os

pretrain_model_path = '/data/LPJ/ICML25/all_checkpoints/pretrain_qformer_havenqwen_using_1989_57_without_lora/v2_cleaned_graph_20epoch_separate_lr_gnn2e3_qformer5e4/pretrain_qformer_havenqwen_using_1989_57_without_lora_v2_cleaned_graph_20epoch_separate_lr_gnn2e3_qformer5e4.ckpt'
pretrain_model = torch.load(pretrain_model_path, map_location='cpu')
new_gnn_path = '/data/LPJ/ICML25/all_checkpoints/graph_tower/pretrain_gnn_qformer_havenqwen_using_1989_57_without_lora/v2_clean_graph_20epoch_separate_lr_gnn2e3_qformer5e4'
new_gnn_path = os.path.join(new_gnn_path, 'clip_gt_arxiv')
os.makedirs(new_gnn_path, exist_ok=True)
new_projector_path = '/data/LPJ/ICML25/all_checkpoints/projector/pretrain_gnn_qformer_havenqwen_using_1989_57_without_lora/v2_clean_graph_20epoch_separate_lr_gnn2e3_qformer5e4'
os.makedirs(new_projector_path, exist_ok=True)

new_gnn_weights_path = os.path.join(new_gnn_path, 'model_weights.pkl')

In [2]:
orig_gnn_path = '/data/LPJ/GraphGPT/clip_gt_arxiv/clip_gt_arxiv_pub.pkl'
orig_path = '/data/LPJ/GraphGPT/clip_gt_arxiv'
orig_gnn = torch.load(orig_gnn_path, map_location='cpu')
##copy config.json from orig_gnn_path to new_gnn_path
import shutil
shutil.copyfile(os.path.join(orig_path, 'config.json'), os.path.join(new_gnn_path, 'config.json'))


'/data/LPJ/ICML25/all_checkpoints/graph_tower/pretrain_gnn_qformer_havenqwen_using_1989_57_without_lora/v2_clean_graph_20epoch_separate_lr_gnn2e3_qformer5e4/clip_gt_arxiv/config.json'

In [3]:
new_graph = {}
for key in pretrain_model['state_dict']:
    if 'graph_tower' in key:
        index = key.find('graph_tower')
        new_key = "gnn" + key[index + len('graph_tower'):]
        new_graph[new_key] = pretrain_model['state_dict'][key]

In [4]:
for key in orig_gnn:
    if key not in new_graph:
        new_graph[key] = orig_gnn[key]

In [5]:
torch.save(new_graph, new_gnn_weights_path)

In [6]:
# from graphgpt.model.GraphLlama import Qformer_proj
# qfomer = Qformer_proj(graph_hidden_size=128, num_query_token=24, bert_name="/data/LPJ/bert/bert-L12-H128-uncased", cross_attention_freq=2, llm_hidden_size=4096)


In [7]:
new_projector = {}
for key in pretrain_model['state_dict']:
    if 'graph_projector' in key:
        index = key.find('graph_projector')
        new_key = key[index + len('graph_projector') + 1:]
        new_projector[new_key] = pretrain_model['state_dict'][key]

In [8]:
torch.save(new_projector, os.path.join(new_projector_path, 'projector.bin'))

In [9]:
# import torch
# new_projector_path = '/data/LPJ/ICML25/all_checkpoints/projector/pretrain_separate_lr_gnn8e3_projector3e4_without_lora/projector.bin'
# new_projector = torch.load(new_projector_path, map_location='cpu')

In [10]:
# new_projector

In [11]:
# pretrain_model['state_dict']